In [ ]:
!pip install TTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from TTS.api import TTS # needs Python 3.11 or older
from ask_lisa import ask_lisa

In [ ]:
base_dir = "asr_bundestag_clean"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"

In [ ]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

In [ ]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

In [ ]:
word_count = {}
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()



In [ ]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if count > 30 and count < 60:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

In [ ]:
def generate_qa(transcript):
    instruction = "Generiere aus dem folgenden Transskript ein Frage und Antwort Paar. Die Frage soll einfach sein und die Antwort nur ein Wort sein:\n"
    #qa = ask_lisa(instruction + transcript)
    return "Frage", "Antwort"

In [ ]:
tts = TTS("tts_models/de/thorsten/vits", progress_bar=False)
def to_speech(text):
    tts.tts_to_file(text=text, file_path="output.wav")

In [ ]:
keys = list(mapping.keys())
     
for i in range(1):
    key = random.choice(keys)
    print(mapping[key][1])
    question, answer = generate_qa(mapping[key][1])
    to_speech(question)
    